# 36c3 Workshop: Introduction to Natural Language Processing

While there are thousands of things we could do, we will limit ourselves to a couple of simple NLP tasks. First, we will implement some of them in pure Python before moving to spaCy, a very robust and widely used NLP library. The code in this notebook is not optimized/writted for production!

In [448]:
from pathlib import Path
import re
import random
from collections import Counter, defaultdict
from difflib import SequenceMatcher

import spacy
import en_core_web_sm

# DIY NLP

## 1. Reading and Tokenizing a Corpus

In [39]:
# Read file and skip description; change case to lowercase
with Path('demo-data/BROWN-a1-a44-modified.txt') as corpus_file:
    corpus = corpus_file.read_text()[350:].lower()

In [43]:
corpus[0:50]

'city controller alexander hemphill charged tuesday'

In [440]:
def tokenize(text):
    return re.split('\s+', text)

In [410]:
# Tokenization
tokens = tokenize(corpus)

In [70]:
tokens

['Ingo', 'has', 'a', 'house', 'in', 'Heidelberg']

## 2. Frequency List & Vocabulary

In [76]:
def frequency_list(tokens):
    return Counter(tokens)

In [392]:
frequency_list(tokens).most_common()

[('the', 6367),
 ('of', 2858),
 ('and', 2164),
 ('to', 2136),
 ('a', 2125),
 ('in', 2013),
 ('for', 965),
 ('that', 810),
 ('is', 720),
 ('was', 714),
 ('on', 681),
 ('he', 637),
 ('at', 633),
 ('with', 566),
 ('be', 523),
 ('as', 519),
 ('by', 501),
 ('his', 426),
 ('it', 426),
 ('will', 386),
 ('from', 353),
 ('are', 319),
 ('an', 311),
 ('this', 309),
 ('--', 300),
 ('has', 299),
 ('had', 280),
 ('but', 276),
 ('they', 265),
 ('who', 265),
 ('have', 264),
 ('were', 255),
 ('not', 252),
 ('mrs.', 252),
 ('said', 249),
 ('would', 244),
 ('new', 237),
 ('which', 234),
 ('their', 231),
 ('been', 211),
 ('its', 201),
 ('one', 196),
 ('there', 182),
 ('i', 176),
 ('last', 175),
 ('more', 174),
 ('or', 171),
 ('all', 171),
 ('mr.', 170),
 ('two', 169),
 ('when', 167),
 ('other', 160),
 ('up', 155),
 ('after', 151),
 ('first', 148),
 ('about', 143),
 ('out', 140),
 ('than', 137),
 ('state', 135),
 ('also', 125),
 ('president', 125),
 ('no', 117),
 ('into', 115),
 ('over', 115),
 ('her', 114

In [406]:
vocabulary = set(tokens)

## 3. N-Grams

In [72]:
def n_grams(tokens, n):
    n_grams = zip(*[tokens[i:] for i in range(n)])
    return [n_gram for n_gram in n_grams]

In [341]:
n_grams(tokens, 3)

[('city', 'controller', 'alexander'),
 ('controller', 'alexander', 'hemphill'),
 ('alexander', 'hemphill', 'charged'),
 ('hemphill', 'charged', 'tuesday'),
 ('charged', 'tuesday', 'that'),
 ('tuesday', 'that', 'the'),
 ('that', 'the', 'bids'),
 ('the', 'bids', 'on'),
 ('bids', 'on', 'the'),
 ('on', 'the', 'frankford'),
 ('the', 'frankford', 'elevated'),
 ('frankford', 'elevated', 'repair'),
 ('elevated', 'repair', 'project'),
 ('repair', 'project', 'were'),
 ('project', 'were', 'rigged'),
 ('were', 'rigged', 'to'),
 ('rigged', 'to', 'the'),
 ('to', 'the', 'advantage'),
 ('the', 'advantage', 'of'),
 ('advantage', 'of', 'a'),
 ('of', 'a', 'private'),
 ('a', 'private', 'contracting'),
 ('private', 'contracting', 'company'),
 ('contracting', 'company', 'which'),
 ('company', 'which', 'had'),
 ('which', 'had', 'an'),
 ('had', 'an', 'inside'),
 ('an', 'inside', 'track'),
 ('inside', 'track', 'with'),
 ('track', 'with', 'the'),
 ('with', 'the', 'city.'),
 ('the', 'city.', 'estimates'),
 ('cit

## 4. A Very Simple N-Gram (Trigram) Language Model

The goal is to predict the/a next word in a sequence of words. We will be implementing the most basic (n-gram-based) language model possible.

In [251]:
trigrams = n_grams(tokens, 3)
model = defaultdict(lambda: defaultdict(lambda: 0))

for t0, t1, t2 in trigrams:
    model[(t0, t1)][t2] += 1

In [116]:
model['i', 'have']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'to': 2,
             'enjoyed': 1,
             'a': 1,
             'ever': 2,
             'the': 1,
             'marveled': 1,
             'never': 1,
             'said': 1})

In [252]:
for t0t1 in model:
    count = float(sum(model[t0t1].values()))
    for t2 in model[t0t1]:
        model[t0t1][t2] /= count

In [294]:
model['and', 'we']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'got': 0.14285714285714285,
             'have': 0.14285714285714285,
             'didnt': 0.14285714285714285,
             'must': 0.14285714285714285,
             'could': 0.14285714285714285,
             'are': 0.14285714285714285,
             'lost': 0.14285714285714285})

### 4.1 Generate Text

In [361]:
def generate_text(model, starting_tokens, max_length, threshold=0.1, verbose=False):
    
    text = starting_tokens
    
    while not len(text) > max_length:
        candidates = []
        for token in model[tuple(text[-2:])].keys():
            if model[tuple(text[-2:])][token] >= threshold:
                candidates.append(token)
        
        try:
            random_candidate = random.choice(candidates)              
            text.append(random_candidate)
            
            if verbose:
                print(f'* "{" ".join(text)}" could be continued with {candidates} --> {random_candidate}')
            
        except IndexError:
            # There are no valid candidates anymore
            break
                
        
    return text

In [381]:
generate_text(model, ['the', 'person'], 5, 0.05, verbose=False)

['the', 'person', 'who', 'believes', 'in', 'dividing']

### 4.2 Crude Spelling Correction

#### a) Vocabulary Approach

In [463]:
def find_closest_match(vocabulary, search):
    best_match = [0, '']
    
    for word in vocabulary:
        ratio = SequenceMatcher(None, search, word).ratio()
        if ratio > best_match[0]:
            best_match = [ratio, word]
            
    return best_match
        

In [468]:
find_closest_match(vocabulary, 'girk')

[0.75, 'girl']

In [520]:
def correct_vocab_approach(text):
    wrong = tokenize(text.lower())
    corrected = []

    for token in wrong:
        if token in vocabulary:
            corrected.append(token)
        else:
            corrected.append(find_closest_match(vocabulary, token)[1])
    
    return ' '.join(corrected)

#### b) Vocabulary & Model Appproach

In [529]:
def correct_vocabmodel_approach(model, text):
    wrong = tokenize(text.lower())
    corrected = []

    for i, token in enumerate(wrong):
        if token in vocabulary:
            corrected.append(token)
        else:
            candidates = list(model[(wrong[i-2], wrong[i-1])])
            if len(candidates) > 0:
                corrected.append(find_closest_match(candidates, token)[1])
            else:
                corrected.append('X')
                    
    return ' '.join(corrected)

In [548]:
wrong = 'The man whp had a beautiful cat.'

print(f'Approach A: {correct_vocab_approach(wrong)}')
print(f'Approach B: {correct_vocabmodel_approach(model, wrong)}')

Approach A: the man whip had a beautiful capt.
Approach B: the man who had a beautiful X


# spaCy

In [2]:
nlp = en_core_web_sm.load()

In [45]:
doc = nlp(corpus)